In [72]:
import pandas as pd
import numpy as np
from datetime import datetime
import statsmodels.api as sm
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split

allfiles1 = {'gopatriots' : 26232}
allfiles={'gohawks':188136,'gopatriots':26232,'nfl':259024,'patriots':489713,'sb49':826951,'superbowl':1348767}

for (file,count) in allfiles.iteritems():
    print "-----------------------",file,"---------------------------"
    csv_df = pd.read_csv("csv/"+file+".csv")
    df = pd.DataFrame(index=range(count), columns=['date','tweets','retweets','followers','maxfollowers'])
    i=0
    for row in csv_df.itertuples(index=True):
        date = datetime.fromtimestamp(getattr(row,"citation_date"))
        df.set_value(i, 'date', date)
        df.set_value(i, 'tweets', getattr(row,"tweet_count"))
        df.set_value(i, 'retweets', getattr(row,"retweetcount"))
        df.set_value(i, 'followers', getattr(row,"follower_count"))
        df.set_value(i, 'maxfollowers', getattr(row,"follower_count"))
        i=i+1
            
    df = df.set_index('date')
    #print df.head()
    hour_data = df.groupby('date').resample('H')
    x = np.zeros((len(hour_data),5))
    y = np.zeros((len(hour_data)))
    
    for j,(interval,group) in enumerate(hour_data):
        x[j,0] = group.tweets.sum()
        x[j,1] = group.retweets.sum()
        x[j,2] = group.followers.sum()
        x[j,3] = group.maxfollowers.max()
        x[j,4] = interval.hour
        
        y[j] = group.tweets.sum()
    
    
    x = np.nan_to_num(x[:-1])
    y = y[1:]
    print x.shape
    
    x = sm.add_constant(x) 
    model = sm.OLS(y,x)
    result = model.fit()
    yhat = result.predict(x)
    rmse = np.sqrt(mean_squared_error(y,yhat))
    print "RMSE for ",file,": ",rmse
    print result.summary()
    print "--------------------------------------------------------"
    

----------------------- superbowl ---------------------------
(586L, 5L)
RMSE for  superbowl :  8003.55892264
                            OLS Regression Results                            
Dep. Variable:                      y   R-squared:                       0.802
Model:                            OLS   Adj. R-squared:                  0.800
Method:                 Least Squares   F-statistic:                     470.4
Date:                Sun, 11 Mar 2018   Prob (F-statistic):          2.21e-201
Time:                        13:49:24   Log-Likelihood:                -6098.3
No. Observations:                 586   AIC:                         1.221e+04
Df Residuals:                     580   BIC:                         1.223e+04
Df Model:                           5                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
---------------------